<a href="https://colab.research.google.com/github/bluebluer/hello-wolrd/blob/master/fastNLP%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastNLP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 295 kB 7.1 MB/s 
  Created wheel for fastNLP: filename=FastNLP-0.7.0-py3-none-any.whl size=364746 sha256=bb4fe7839d6f1d0d243ee5b40218432eeb24bc0a2037f0082065e885e421fbfc
  Stored in directory: /root/.cache/pip/wheels/eb/80/db/f206d6f4481007868937e789999822fc879aad6e44ddcbcafa
Successfully built fastNLP


In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from fastNLP.io import ChnSentiCorpLoader

loader = ChnSentiCorpLoader()        # 初始化一个中文情感分类的loader
data_dir = loader.download()         # 这一行代码将自动下载数据到默认的缓存地址, 并将该地址返回
data_bundle = loader.load(data_dir)  # 这一行代码将从{data_dir}处读取数据至DataBundle

  1%|          | 16.4k/1.76M [00:00<00:17, 101kB/s]

http://download.fastnlp.top/dataset/chn_senti_corp.zip not found in cache, downloading to /tmp/tmp3k9qsmln


100%|██████████| 1.76M/1.76M [00:01<00:00, 1.50MB/s]


Finish download from http://download.fastnlp.top/dataset/chn_senti_corp.zip
Copy file to /root/.fastNLP/dataset/chn_senti_corp


In [4]:
print(data_bundle)

In total 3 datasets:
	dev has 1200 instances.
	test has 1200 instances.
	train has 9600 instances.



In [5]:
print(data_bundle.get_dataset('train')[:2])  # 查看Train集前两个sample

+-------------------------------------------+--------+
| raw_chars                                 | target |
+-------------------------------------------+--------+
| 选择珠江花园的原因就是方便，有电动扶梯... | 1      |
| 15.4寸笔记本的键盘确实爽，基本跟台式机... | 1      |
+-------------------------------------------+--------+


In [6]:
from fastNLP.io import ChnSentiCorpPipe

pipe = ChnSentiCorpPipe()
data_bundle = pipe.process(data_bundle)  # 所有的Pipe都实现了process()方法，且输入输出都为DataBundle类型

print(data_bundle)  # 打印data_bundle，查看其变化

In total 3 datasets:
	dev has 1200 instances.
	test has 1200 instances.
	train has 9600 instances.
In total 2 vocabs:
	chars has 4409 entries.
	target has 2 entries.



In [7]:
print(data_bundle.get_dataset('train')[:2])

+-------------------------+--------+------------------------+---------+
| raw_chars               | target | chars                  | seq_len |
+-------------------------+--------+------------------------+---------+
| 选择珠江花园的原因就... | 0      | [338, 464, 1400, 78... | 106     |
| 15.4寸笔记本的键盘确... | 0      | [50, 133, 20, 135, ... | 56      |
+-------------------------+--------+------------------------+---------+


In [8]:
char_vocab = data_bundle.get_vocab('chars')
print(char_vocab)

Vocabulary(['选', '择', '珠', '江', '花']...)


In [9]:
from fastNLP.embeddings import StaticEmbedding

word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name='cn-char-fastnlp-100d')

  0%|          | 16.4k/3.70M [00:00<00:43, 84.5kB/s]

http://download.fastnlp.top/embedding/cn_char_fastnlp_100d.zip not found in cache, downloading to /tmp/tmp0iz8vge2


100%|██████████| 3.70M/3.70M [00:01<00:00, 2.34MB/s]


Finish download from http://download.fastnlp.top/embedding/cn_char_fastnlp_100d.zip
Copy file to /root/.fastNLP/embedding/cn_char_fastnlp_100d
Found 4321 out of 4409 words in the pre-training embedding.


In [10]:
from torch import nn
from fastNLP.modules import LSTM
import torch

# 定义模型
class BiLSTMMaxPoolCls(nn.Module):
    def __init__(self, embed, num_classes, hidden_size=400, num_layers=1, dropout=0.3):
        super().__init__()
        self.embed = embed

        self.lstm = LSTM(self.embed.embedding_dim, hidden_size=hidden_size//2, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, chars, seq_len):  # 这里的名称必须和DataSet中相应的field对应，比如之前我们DataSet中有chars，这里就必须为chars
        # chars:[batch_size, max_len]
        # seq_len: [batch_size, ]
        chars = self.embed(chars)
        outputs, _ = self.lstm(chars, seq_len)
        outputs = self.dropout_layer(outputs)
        outputs, _ = torch.max(outputs, dim=1)
        outputs = self.fc(outputs)

        return {'pred':outputs}  # [batch_size,], 返回值必须是dict类型，且预测值的key建议设为pred

# 初始化模型
model = BiLSTMMaxPoolCls(word2vec_embed, len(data_bundle.get_vocab('target')))

In [11]:
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam
from fastNLP import AccuracyMetric

loss = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
metric = AccuracyMetric()
device = 0 if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快

trainer = Trainer(train_data=data_bundle.get_dataset('train'), model=model, loss=loss,
                  optimizer=optimizer, batch_size=32, dev_data=data_bundle.get_dataset('dev'),
                  metrics=metric, device=device)
trainer.train()  # 开始训练，训练完成之后默认会加载在dev上表现最好的模型

# 在测试集上测试一下模型的性能
from fastNLP import Tester
print("Performance on test is:")
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
tester.test()

input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	chars: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 106]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2022-06-01-13-05-29-367351


  0%|          | 0/3000 [00:00<?, ?it/s, loss:{0:<6.5f}]

  0%|          | 0/38 [00:00<?, ?it/s]

Evaluate data in 0.48 seconds!
Evaluation on dev at Epoch 1/10. Step:300/3000: 
AccuracyMetric: acc=0.811667



  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

Evaluate data in 0.45 seconds!
Evaluation on dev at Epoch 10/10. Step:3000/3000: 
AccuracyMetric: acc=0.9025

Reloaded the best model.

In Epoch:9/Step:2700, got best dev performance:
AccuracyMetric: acc=0.911667
Performance on test is:


  0%|          | 0/19 [00:00<?, ?it/s]

Evaluate data in 0.32 seconds!
[tester] 
AccuracyMetric: acc=0.9025


{'AccuracyMetric': {'acc': 0.9025}}

In [12]:
# 使用bert进行文本分类
# 只需要切换一下Embedding即可
from fastNLP.embeddings import BertEmbedding

# 这里为了演示一下效果，所以默认Bert不更新权重
bert_embed = BertEmbedding(char_vocab, model_dir_or_name='cn', auto_truncate=True, requires_grad=False)
model = BiLSTMMaxPoolCls(bert_embed, len(data_bundle.get_vocab('target')))


import torch
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam
from fastNLP import AccuracyMetric

loss = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=2e-5)
metric = AccuracyMetric()
device = 0 if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快

trainer = Trainer(train_data=data_bundle.get_dataset('train'), model=model, loss=loss,
                  optimizer=optimizer, batch_size=16, dev_data=data_bundle.get_dataset('test'),
                  metrics=metric, device=device, n_epochs=3)
trainer.train()  # 开始训练，训练完成之后默认会加载在dev上表现最好的模型

# 在测试集上测试一下模型的性能
from fastNLP import Tester
print("Performance on test is:")
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
tester.test()

  0%|          | 16.4k/412M [00:00<1:07:21, 102kB/s]

http://download.fastnlp.top/embedding/bert-chinese-wwm.zip not found in cache, downloading to /tmp/tmp34km40oh


100%|██████████| 412M/412M [02:43<00:00, 2.52MB/s]


Finish download from http://download.fastnlp.top/embedding/bert-chinese-wwm.zip
Copy file to /root/.fastNLP/embedding/bert-chinese-wwm
loading vocabulary file /root/.fastNLP/embedding/bert-chinese-wwm/vocab.txt
Load pre-trained BERT parameters from file /root/.fastNLP/embedding/bert-chinese-wwm/chinese_wwm_pytorch.bin.
Bert Model will return 1 layers (layer-0 is embedding result): [-1]
input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	chars: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 106]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2022-06-01-13-10-47-677905


  0%|          | 0/1800 [00:00<?, ?it/s, loss:{0:<6.5f}]

...

  0%|          | 0/75 [00:00<?, ?it/s]

Evaluate data in 27.79 seconds!
Evaluation on dev at Epoch 3/3. Step:1800/1800: 
AccuracyMetric: acc=0.909167

Reloaded the best model.

In Epoch:2/Step:1200, got best dev performance:
AccuracyMetric: acc=0.910833
Performance on test is:


  0%|          | 0/19 [00:00<?, ?it/s]

Evaluate data in 38.39 seconds!
[tester] 
AccuracyMetric: acc=0.91


{'AccuracyMetric': {'acc': 0.91}}

In [13]:
#基于词进行文本分类
from fastNLP.io import ChnSentiCorpLoader

loader = ChnSentiCorpLoader()        # 初始化一个中文情感分类的loader
data_dir = loader.download()         # 这一行代码将自动下载数据到默认的缓存地址, 并将该地址返回


In [14]:
import os
from fastNLP import DataSet, Instance
from fastNLP.io import DataBundle


def read_file_to_dataset(fp):
    ds = DataSet()
    with open(fp, 'r') as f:
        f.readline()  # 第一行是title名称，忽略掉
        for line in f:
            line = line.strip()
            target, chars = line.split('\t')
            ins = Instance(target=target, raw_chars=chars)
            ds.append(ins)
    return ds

data_bundle = DataBundle()
for name in ['train.tsv', 'dev.tsv', 'test.tsv']:
    fp = os.path.join(data_dir, name)
    ds = read_file_to_dataset(fp)
    data_bundle.set_dataset(name=name.split('.')[0], dataset=ds)

print(data_bundle)  # 查看以下数据集的情况
# In total 3 datasets:
#    train has 9600 instances.
#    dev has 1200 instances.
#    test has 1200 instances.

In total 3 datasets:
	train has 9600 instances.
	dev has 1200 instances.
	test has 1200 instances.



In [16]:
! pip install fastHan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.5 MB/s 


In [17]:
from fastHan import FastHan
from fastNLP import Vocabulary

model=FastHan()
# model.set_device('cuda')  # 可以注视掉这一行增加速度

# 定义分词处理操作
def word_seg(ins):
    raw_chars = ins['raw_chars']
    # 由于有些句子比较长，我们只截取前128个汉字
    raw_words = model(raw_chars[:128], target='CWS')[0]
    return raw_words

for name, ds in data_bundle.iter_datasets():
    # apply函数将对内部的instance依次执行word_seg操作，并把其返回值放入到raw_words这个field
    ds.apply(word_seg, new_field_name='raw_words')
    # 除了apply函数，fastNLP还支持apply_field, apply_more(可同时创建多个field)等操作
    # 同时我们增加一个seq_len的field
    ds.add_seq_len('raw_words')

vocab = Vocabulary()

# 对raw_words列创建词表, 建议把非训练集的dataset放在no_create_entry_dataset参数中
# 也可以通过add_word(), add_word_lst()等建立词表，请参考http://www.fastnlp.top/docs/fastNLP/tutorials/tutorial_2_vocabulary.html
vocab.from_dataset(data_bundle.get_dataset('train'), field_name='raw_words',
                no_create_entry_dataset=[data_bundle.get_dataset('dev'),
                                            data_bundle.get_dataset('test')])

# 将建立好词表的Vocabulary用于对raw_words列建立词表，并把转为序号的列存入到words列
vocab.index_dataset(data_bundle.get_dataset('train'), data_bundle.get_dataset('dev'),
                data_bundle.get_dataset('test'), field_name='raw_words', new_field_name='words')

# 建立target的词表，target的词表一般不需要padding和unknown
target_vocab = Vocabulary(padding=None, unknown=None)
# 一般情况下我们可以只用训练集建立target的词表
target_vocab.from_dataset(data_bundle.get_dataset('train'), field_name='target')
# 如果没有传递new_field_name, 则默认覆盖原词表
target_vocab.index_dataset(data_bundle.get_dataset('train'), data_bundle.get_dataset('dev'),
                data_bundle.get_dataset('test'), field_name='target')

# 我们可以把词表保存到data_bundle中，方便之后使用
data_bundle.set_vocab(field_name='words', vocab=vocab)
data_bundle.set_vocab(field_name='target', vocab=target_vocab)

# 我们把words和target分别设置为input和target，这样它们才会在训练循环中被取出并自动padding, 有关这部分更多的内容参考
#  http://www.fastnlp.top/docs/fastNLP/tutorials/tutorial_6_datasetiter.html
data_bundle.set_target('target')
data_bundle.set_input('words')  # DataSet也有这两个接口
# 如果某些field，您希望它被设置为target或者input，但是不希望fastNLP自动padding或需要使用特定的padding方式，请参考
#  http://www.fastnlp.top/docs/fastNLP/fastNLP.core.dataset.html

print(data_bundle.get_dataset('train')[:2])  # 我们可以看一下当前dataset的内容

# +--------+-----------------------+-----------------------+----------------------+
# | target | raw_chars             | raw_words             | words                |
# +--------+-----------------------+-----------------------+----------------------+
# | 0      | 选择珠江花园的原因... | ['选择', '珠江', ...  | [2, 3, 4, 5, 6, 7... |
# | 0      | 15.4寸笔记本的键盘... | ['15.4', '寸', '笔... | [71, 72, 73, 74, ... |
# +--------+-----------------------+-----------------------+----------------------+

# 由于之后需要使用之前定义的BiLSTMMaxPoolCls模型，所以需要将words这个field修改为chars
data_bundle.rename_field('words', 'chars')

  0%|          | 16.4k/144M [00:00<27:17, 88.0kB/s]

http://212.129.155.247/fasthan/fasthan_base.zip not found in cache, downloading to /tmp/tmpi9g0p4he


100%|██████████| 144M/144M [00:57<00:00, 2.52MB/s]


Finish download from http://212.129.155.247/fasthan/fasthan_base.zip
Copy file to /root/.fastNLP/fasthan/fasthan_base
loading vocabulary file /root/.fastNLP/fasthan/fasthan_base/vocab.txt
Load pre-trained BERT parameters from file /root/.fastNLP/fasthan/fasthan_base/model.bin.
+--------+---------------------+--------------------+---------+--------------------+
| target | raw_chars           | raw_words          | seq_len | words              |
+--------+---------------------+--------------------+---------+--------------------+
| 1      | 选择珠江花园的原... | ['选择', '珠江'... | 69      | [220, 8212, 153... |
| 1      | 15.4寸笔记本的键... | ['15.4', '寸', ... | 34      | [4384, 775, 358... |
+--------+---------------------+--------------------+---------+--------------------+


In total 3 datasets:
	train has 9600 instances.
	dev has 1200 instances.
	test has 1200 instances.
In total 2 vocabs:
	target has 2 entries.
	chars has 27341 entries.

In [19]:
print(data_bundle.get_vocab('chars'))
# Vocabulary([选择, 珠江, 花园, 的, 原因]...)

Vocabulary(['选择', '珠江', '花园', '的', '原因']...)
